In [4]:
import pandas as pd

df_clickBait=pd.read_csv("annotated\combined\csv\main.csv")

In [6]:
df_clickBait

,title,label,label_score
0,"Masuk Radar Pilwalkot Medan, Menantu Jokowi Be...",non-clickbait,0
1,Malaysia Sudutkan RI: Isu Kabut Asap hingga In...,non-clickbait,0
2,Viral! Driver Ojol di Bekasi Antar Pesanan Mak...,clickbait,1
3,"Kemensos Salurkan Rp 7,3 M bagi Korban Kerusuh...",non-clickbait,0
4,"Terkait Mayat Bayi Mengenaskan di Tangerang, S...",non-clickbait,0
...,...,...,...
14995,"Tolak RUU Pertanahan, Ribuan Petani Siap Gelar...",non-clickbait,0
14996,Ada Niat Tambah Momongan Tanpa Ikut Program Ha...,clickbait,1
14997,"Beredar Isu Internet Papua Diblokir Lagi, Telk...",non-clickbait,0
14998,"TXT Akan Segera Comeback, Soobin Akui Gatal I...",clickbait,1


In [21]:
import os

In [19]:
# import modules & set up logging
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 
sentences = [['first', 'sentence'], ['second', 'sentence']]
# train word2vec on the two sentences
model = gensim.models.Word2Vec(sentences, min_count=1)

2021-12-01 04:38:59,912 : INFO : collecting all words and their counts
2021-12-01 04:38:59,913 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-12-01 04:38:59,914 : INFO : collected 3 word types from a corpus of 4 raw words and 2 sentences
2021-12-01 04:38:59,915 : INFO : Creating a fresh vocabulary
2021-12-01 04:38:59,918 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 3 unique words (100.0%% of original 3, drops 0)', 'datetime': '2021-12-01T04:38:59.917304', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'prepare_vocab'}
2021-12-01 04:38:59,919 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 4 word corpus (100.0%% of original 4, drops 0)', 'datetime': '2021-12-01T04:38:59.919299', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.

In [28]:
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield line.split()
 
sentences = MySentences(df_clickBait['title'][0]) # a memory-friendly iterator
model = gensim.models.Word2Vec(sentences)

2021-12-01 04:42:04,429 : INFO : collecting all words and their counts


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Masuk Radar Pilwalkot Medan, Menantu Jokowi Bertemu DPW NasDem Sumut'

In [13]:
#Splitting the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(df_clickBait['title'], df_clickBait['label_score'], test_size = 0.3, random_state = 100)

X_train

5220     Menanti Suku Bunga The Fed, Harga Emas Naik Tipis
4819                      4 Fakta soal Sakitnya BJ Habibie
9065     Black Knight Akan Buktikan Keberadaan Mutan di...
8696                 The press morally indebted to Habibie
5571     Kapendam IV Diponegoro: Tindakan Represif Praj...
                               ...                        
12119    Giliran Nono Sampono Kunjungi Sabam sebagai Se...
8039     Suporter Thailand Kecewa SUGBK Sepi dari Supor...
14147    AMM Desak Kematian Mahasiswa Kendari Diusut: K...
6936     Quartararo Bahagia Bisa Bertarung dengan Marqu...
5640                 Kendaraan Tambang Diminta Gunakan B20
Name: title, Length: 10500, dtype: object

In [15]:
# Menjalankan algoritma PCA
from sklearn.decomposition import PCA
pca = PCA(n_components = None)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
hasil_variance = pca.explained_variance_ratio_ # Proses pengecekan variance

ValueError: could not convert string to float: 'Menanti Suku Bunga The Fed, Harga Emas Naik Tipis'

In [ ]:
# Proses pemilihan PCs
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
pca = PCA(n_components = 2)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
hasil_variance2 = pca.explained_variance_ratio_
 
# Menjalankan algoritma logistic regression ke training set
from sklearn.linear_model import LogisticRegression
mesin_klasifikasi = LogisticRegression(random_state = 0)
mesin_klasifikasi.fit(X_train, y_train)
 
# Memprediksi test set berdasakan model logistic regression
y_pred = mesin_klasifikasi.predict(X_test)
 
# Membuat confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
 
# Visualisasi Training Set
from matplotlib.colors import ListedColormap
X_set, y_set = X_train, y_train
X1, X2 = np.meshgrid(np.arange(start = X_set[:, 0].min() - 1, stop = X_set[:, 0].max() + 1, step = 0.01),
                     np.arange(start = X_set[:, 1].min() - 1, stop = X_set[:, 1].max() + 1, step = 0.01))
plt.contourf(X1, X2, mesin_klasifikasi.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),
             alpha = 0.75, cmap = ListedColormap(('red', 'green', 'blue')))
plt.xlim(X1.min(), X1.max())
plt.ylim(X2.min(), X2.max())
for i, j in enumerate(np.unique(y_set)):
    plt.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1],
                c = ListedColormap(('red', 'green', 'blue'))(i), label = j)
plt.title('Logistic Regression (Training set)')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.legend()
plt.show()
 
# Visualisasi Test Set
from matplotlib.colors import ListedColormap
X_set, y_set = X_test, y_test
X1, X2 = np.meshgrid(np.arange(start = X_set[:, 0].min() - 1, stop = X_set[:, 0].max() + 1, step = 0.01),
                     np.arange(start = X_set[:, 1].min() - 1, stop = X_set[:, 1].max() + 1, step = 0.01))
plt.contourf(X1, X2, mesin_klasifikasi.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),
             alpha = 0.75, cmap = ListedColormap(('red', 'green', 'blue')))
plt.xlim(X1.min(), X1.max())
plt.ylim(X2.min(), X2.max())
for i, j in enumerate(np.unique(y_set)):
    plt.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1],
                c = ListedColormap(('red', 'green', 'blue'))(i), label = j)
plt.title('Logistic Regression (Test set)')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.legend()
plt.show()